In [ ]:
import os
import glob
import re
import uuid
import torch
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer, AutoModelForMaskedLM, BitsAndBytesConfig
from qdrant_client import QdrantClient, models
from qdrant_client.models import PointStruct, SparseVector, Distance, VectorParams, SparseVectorParams
from langchain_text_splitters import MarkdownHeaderTextSplitter


# Path Folder Markdown
FOLDER_PATH = r"C:\\Users\\Ilmu Komputer\\OneDrive\\Desktop\\portofolio\\RAG\\defi-rag-agent\\src\\evaluation\\data_md\\real"


DENSE_MODEL_ID = "Qwen/Qwen3-Embedding-4B"
SPARSE_MODEL_ID = "naver/splade-v3"
QDRANT_PATH = "./qdrant_custom_db_qwen"
COLLECTION_NAME = "hybrid_qwen_splade_optimized"


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on: {device}")

# Konfigurasi Kuantisasi 4-bit
bnb_config = None
if device == "cuda":
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )


#CLASS EMBEDDER (DENSE + SPARSE)
def last_token_pool(last_hidden_states: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
    """Helper function khusus untuk Qwen/GTE embedding agar ambil token terakhir"""
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

class CustomEmbedder:
    def __init__(self):
        print("⏳ Loading Dense Model (Qwen 4-bit)...")
        self.dense_tokenizer = AutoTokenizer.from_pretrained(DENSE_MODEL_ID, trust_remote_code=True)
        self.dense_model = AutoModel.from_pretrained(
            DENSE_MODEL_ID,
            trust_remote_code=True,
            quantization_config=bnb_config,
            device_map="auto" if device == "cuda" else None,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32
        )
        if device == "cpu": self.dense_model.to("cpu")

        print("⏳ Loading Sparse Model (Splade v3)...")
        self.sparse_tokenizer = AutoTokenizer.from_pretrained(SPARSE_MODEL_ID)
        self.sparse_model = AutoModelForMaskedLM.from_pretrained(SPARSE_MODEL_ID)
        self.sparse_model.to(device)

    def get_dense_vector(self, text):
        inputs = self.dense_tokenizer(
            text, max_length=8192, padding=True, truncation=True, return_tensors='pt'
        )
        inputs = {k: v.to(self.dense_model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.dense_model(**inputs)
            embeddings = last_token_pool(outputs.last_hidden_state, inputs['attention_mask'])
            embeddings = F.normalize(embeddings, p=2, dim=1)
            
        return embeddings[0].cpu().tolist()

    def get_sparse_vector(self, text):
        inputs = self.sparse_tokenizer(
            text, return_tensors="pt", padding=True, truncation=True
        )
        inputs = {k: v.to(self.sparse_model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.sparse_model(**inputs)
        logits = outputs.logits[0]
        relu_log = torch.log(1 + torch.relu(logits))
        attention_mask = inputs["attention_mask"][0].unsqueeze(-1)
        weighted_log = relu_log * attention_mask
        max_val, _ = torch.max(weighted_log, dim=0)
        indices = torch.nonzero(max_val).squeeze().cpu().tolist()
        values = max_val[indices].cpu().tolist()
        if isinstance(indices, int):
            indices = [indices]
            values = [values]
            
        return SparseVector(indices=indices, values=values)


#SETUP DATABASE QDRANT
embedder = CustomEmbedder()
print(f"\nMembuka database Qdrant lokal di: {QDRANT_PATH}")
client = QdrantClient(path=QDRANT_PATH)

# Cek apakah collection SUDAH ADA
if not client.collection_exists(COLLECTION_NAME):
    print("Collection belum ada. Membuat Collection baru...")
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config={
            "dense_vector": VectorParams(size=2560, distance=Distance.COSINE) 
        },
        sparse_vectors_config={
            "sparse_vector": SparseVectorParams()
        }
    )
else:
    print(f"Collection '{COLLECTION_NAME}' ditemukan. Mode: APPEND (Menambahkan data).")


#FUNGSI UTILITY (CLEANING & SPLITTING)
def clean_title(filename):
    """Membersihkan nama file dari ekstensi dan timestamp (misal: _250130_133808)"""
    name = os.path.splitext(filename)[0]
    name = re.sub(r'[-_]\d{6,}.*', '', name)
    name = name.replace("-", " ").replace("_", " ")
    return " ".join(name.split())

headers_to_split_on = [("#", "Header 1"), ("##", "Header 2"), ("###", "Header 3")]
md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

md_files = glob.glob(os.path.join(FOLDER_PATH, "*.md"))
print(f"Ditemukan {len(md_files)} file Markdown.")


#PROSES EMBEDDING & UPSERT
points = []
print("Mulai proses embedding...")

# LOOP 1: FILE
for file_index, file_path in enumerate(md_files):
    file_name = os.path.basename(file_path)
    
    # BERSIHKAN JUDUL
    doc_title = clean_title(file_name)
    
    with open(file_path, "r", encoding="utf-8") as f:
        file_content = f.read()
        
    docs = md_splitter.split_text(file_content)
    total_chunks = len(docs)
    
    print(f"\nFile [{file_index+1}/{len(md_files)}]: {file_name}")
    print(f"Clean Title: '{doc_title}'")

    # LOOP 2: CHUNK
    for i, doc in enumerate(docs):
        raw_content = doc.page_content.strip()
        if not raw_content: continue

        # Ambil header hierarchy
        header_keys = [name for _, name in headers_to_split_on]
        found_headers = []
        for key in header_keys:
            if key in doc.metadata:
                found_headers.append(doc.metadata[key])
        context_chain = [doc_title] + found_headers
        breadcrumb_str = " > ".join(context_chain)
        final_text = f"{breadcrumb_str} : {raw_content}"

        print(f"Chunk [{i+1}/{total_chunks}] | {breadcrumb_str}")

        try:
            d_vec = embedder.get_dense_vector(final_text)
            s_vec = embedder.get_sparse_vector(final_text)

            if file_index == 0 and i == 0:
                print(f"Detected Dense Vector Size: {len(d_vec)}")

            metadata = doc.metadata
            metadata["source_file"] = file_name
            metadata["doc_title"] = doc_title
            metadata["chunk_index"] = i
            
            points.append(PointStruct(
                id=str(uuid.uuid4()), 
                vector={
                    "dense_vector": d_vec,
                    "sparse_vector": s_vec
                },
                payload={
                    "text": final_text,
                    "original_content": raw_content,
                    "metadata": metadata
                }
            ))
        except Exception as e:
            print(f"Error embedding chunk: {e}")

# Upload ke Qdrant
if points:
    print(f"\nMengupload {len(points)} points ke Qdrant...")
    BATCH_SIZE = 50
    for i in range(0, len(points), BATCH_SIZE):
        batch = points[i:i+BATCH_SIZE]
        client.upsert(collection_name=COLLECTION_NAME, points=batch)
        print(f"   Saved batch {i} - {i+len(batch)}")
    print("Semua data berhasil diupload!")
else:
    print("Tidak ada data point yang dihasilkan.")



Running on: cuda
⏳ Loading Dense Model (Qwen 4-bit)...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

⏳ Loading Sparse Model (Splade v3)...

Membuka database Qdrant lokal di: ./qdrant_custom_db_qwen
Collection belum ada. Membuat Collection baru...
Ditemukan 13 file Markdown.
Mulai proses embedding...

File [1/13]: Dukungan-Pelaksanaan-Kebijakan-Efisiensi-Belanja-APBN-Tahun-Anggaran-2025pdf.md
Clean Title: 'Dukungan Pelaksanaan Kebijakan Efisiensi Belanja APBN Tahun Anggaran 2025pdf'
Chunk [1/1] | Dukungan Pelaksanaan Kebijakan Efisiensi Belanja APBN Tahun Anggaran 2025pdf > KEMENTERIAN KEUANGAN REPUBLIK INDONESIA > DIREKTORAT JENDERAL PERBENDAHARAAN
Detected Dense Vector Size: 2560

File [2/13]: Dukungan-Pelaksanaan-Kebijakan-Pemerintahpdf.md
Clean Title: 'Dukungan Pelaksanaan Kebijakan Pemerintahpdf'
Chunk [1/1] | Dukungan Pelaksanaan Kebijakan Pemerintahpdf > KEMENTERIAN KEUANGAN REPUBLIK INDONESIA > DIREKTORAT JENDERAL PERBENDAHARAAN

File [3/13]: Efisiensi-KL-TA-2025_250130_133808.md
Clean Title: 'Efisiensi KL TA 2025'
Chunk [1/2] | Efisiensi KL TA 2025 > REPUBLIK INDONESIA
Chunk [

UJI PERTANYAAN

In [1]:
from RAG import rrf_retriever_chain, rrf_retriever_chain_gemma, rrf_retriever_chain_gemma3_12b

async def query_rrf(query, chain):
    print("\n" + "=" * 60)
    print(f"PERTANYAAN: {query}")
    print("=" * 60)

    result = await chain(query)
    ans = result.get('answer', "No answer found")
    source_docs = result.get('source_documents', [])
    doc_contents = []

    print(f"\nJAWABAN:\n{ans}\n")
    print("-" * 60)
    print("KONTEKS SUMBER (TOP RERANKED):")
    print("-" * 60)

    for idx, doc in enumerate(source_docs, 1):
        text = ""
        score = "N/A"
        metadata = {}

        # 1. Handling berbagai format input (Qdrant Point, LangChain Doc, or Dict)
        if hasattr(doc, 'payload'): 
            text = doc.payload.get('text', '')
            score = getattr(doc, 'score', 'N/A')
            metadata = doc.payload.get('metadata', {})
        elif hasattr(doc, 'page_content'):
            text = doc.page_content
            metadata = getattr(doc, 'metadata', {})
            # Ambil score dari metadata reranker jika ada
            score = metadata.get('relevance_score', 'N/A')
        elif isinstance(doc, dict): 
            text = doc.get('text', str(doc))
            score = doc.get('score', 'N/A')
            metadata = doc.get('metadata', {})

        doc_contents.append(text)

        # 2. Cetak ke bawah secara terstruktur
        title = metadata.get('title', 'No Title')
        print(f"DOKUMEN KE-{idx}")
        print(f"   [SKOR]  : {score}")
        print(f"   [JUDUL] : {title}")
        print(f"   [ISI]   : {text}...") 
        print(f"\n" + "." * 30 + "\n") 

    if not doc_contents:
        print("Tidak ada dokumen yang ditemukan.")
    
    print("=" * 60 + "\n")
    
    return {
        'question': query,
        'answer': ans,
        'contexts': doc_contents
    }

Koneksi ke Firebase Firestore berhasil.
Inisialisasi LLM dengan model: meta-llama/llama-4-scout-17b-16e-instruct
Inisialisasi Evaluator dengan model: gpt-4o-mini
Running on: cuda
Loading Dense Model (Qwen 4-bit)...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading Sparse Model (Splade v3)...


In [5]:
q = 'apa tugas PA?'

hasil = await query_rrf(q, rrf_retriever_chain)


PERTANYAAN: apa tugas PA?
[INFO] Reranking 20 dokumen...

JAWABAN:
Menteri Keuangan menetapkan besaran efisiensi anggaran belanja masing-masing Kementerian/Lembaga Tahun Anggaran 2025.

------------------------------------------------------------
KONTEKS SUMBER (TOP RERANKED):
------------------------------------------------------------
DOKUMEN KE-1
   [SKOR]  : 0.026219982653856277
   [JUDUL] : No Title
   [ISI]   : SK Daftar Informasi Informasi Publik Undiksha Tahun 2024 > KEPUTUSAN REKTOR UNIVERSITAS PENDIDIKAN GANESHA > DAFTAR INFORMASI PUBLIK YANG WAJIB DISEDIAKAN DAN DIUMUMKAN SECARA BERKALA : <table>
<thead>
<tr>
<th>NO</th>
<th>Ringkasan Isi Informasi</th>
<th>Pejabat/Unit/Satke
<br/>
r yang Menguasai</th>
<th>Penanggungjawab
<br/>
Pembuatan atau
<br/>
Penerbitan
<br/>
Informasi</th>
<th>Format
<br/>
Informasi yang
<br/>
Tersedia</th>
<th>Jangkawaktu
<br/>
disediakan
<br/>
(berkala)</th>
<th>Jangka
<br/>
Waktu
<br/>
Penyimpanan
<br/>
atau Retensi
<br/>
Arsip</th>
</tr>
</thead

In [6]:
#ini justru RRF gabisa, perlu bantuan rerangker buat retive atau buat query semakin singkat biar fokus ke kode matkul atau bisa juga pake regex buat eksrak biar ga ada latency api
q = 'berapa jumlah sks prodib Akuntansi semester 2?'
hasil = await query_rrf(q, rrf_retriever_chain)


PERTANYAAN: berapa jumlah sks prodib Akuntansi semester 2?
[INFO] Reranking 20 dokumen...

JAWABAN:
Tidak tau. 

(Konteks tidak menyediakan informasi tentang jumlah sks prodi Akuntansi semester 2)

------------------------------------------------------------
KONTEKS SUMBER (TOP RERANKED):
------------------------------------------------------------
DOKUMEN KE-1
   [SKOR]  : 0.005244911182671785
   [JUDUL] : No Title
   [ISI]   : pengumuman beasiswa ganesha 2025 > PENGUMUMAN > A. Persyaratan > (1) Bagi Mahasiswa Tidak Mampu : 1. Terdaftar sebagai mahasiswa aktif (S1,D4,D3) Undiksha pada semester genap Tahun Akademik 2024/2025. Untuk Jenjang S1 paling rendah duduk pada semester II dan paling tinggi pada semester VI. Untuk Diploma IV paling rendah duduk pada semester II dan paling tinggi pada semester VI. Untuk Diploma III paling rendah duduk pada semester II paling tinggi pada semester IV pada semester genap TA 2024/2025.  
2. Melampirkan Kartu Hasil Studi (KHS) serta memiliki Indeks Pr

Pengujian query multihop

In [9]:
#pengujian multihop dimana ga pakai rerangkar malah bilang pak rasben rektornya, ketika pakai v2m3 maupun large bge akhirnya bisa tapi v2m4 butuh 1 menit sedangkan large hanya 6 detik. ada dikasus lain bge large ga bisa jawab

import resources
q = "siapa rektor dari pencipta mu"
q1 = "siapa pencipta mu?"
q2 = "dimana Fieter kuliah?"
q3 = "siapa rektor undiksha?"

hasil = await query_rrf(q1, rrf_retriever_chain)
multihop = await query_rrf(q2, rrf_retriever_chain)
result = await query_rrf(q3, rrf_retriever_chain)

end_multihop = f"Jawab singkat pertanyaan {q} berdasarkan konteks berikut: {hasil.get('answer')}, {multihop.get('answer')}, {result.get('answer')}"
response_llm = await resources.llm.ainvoke(end_multihop)
print(response_llm.content)


PERTANYAAN: siapa pencipta mu?
[INFO] Reranking 20 dokumen...

JAWABAN:
Pencipta saya adalah Fieter, seorang AI Engineer & Research Student.

------------------------------------------------------------
KONTEKS SUMBER (TOP RERANKED):
------------------------------------------------------------
DOKUMEN KE-1
   [SKOR]  : 0.0022031257394701242
   [JUDUL] : No Title
   [ISI]   : pencipta RAG defi > Profil Pencipta, AI Engineer & Research Student, Fieter > 3. Pengalaman Proyek & Riset : - **Intelligent Contract & Invoice Auditor:** Mengembangkan aplikasi ekstraksi data otomatis dari PDF menggunakan AI (FastAPI & Tailwind).
- **Internship UPATIK:** Fokus pada pengembangan dan optimalisasi sistem layanan informasi berbasis AI (RAG system).
- **Klasifikasi Motif Songket Jinengdalem:** Penelitian akademik menggunakan arsitektur MobileNetV2 untuk klasifikasi motif songket.
- **Optimasi Hybrid Search:** Implementasi pencarian gabungan (dense & sparse vector) menggunakan Qdrant dan reranker.
- **

In [ ]:
#pake bge large malah ngasih jawaban rektorat, pake v2m3 baru bener
w = "siapa saja nama pimpinan dari FTK"
result = await query_rrf(w, rrf_retriever_chain)


PERTANYAAN: siapa saja nama pimpinan dari FTK
[INFO] Reranking 20 dokumen...
⏳ Loading Reranker: BAAI/bge-reranker-large


Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))



JAWABAN:
Nama pimpinan dari FTK adalah Rektor Dr. I Nyoman Jampel, M.Pd., Wakil Rektor I Prof. Dr. Ida Bagus Putu Arnyana, M.Si., Wakil Rektor II Prof. Dr. I Wayan Lasmawan, M.Pd., Wakil Rektor III Dr. I Gusti Ngurah Pujawan, M.Kes., dan Wakil Rektor IV Drs. I Wayan Suarnajaya, M.A., Ph.D.

------------------------------------------------------------
KONTEKS SUMBER (TOP RERANKED):
------------------------------------------------------------
DOKUMEN KE-1
   [SKOR]  : 0.9985536932945251
   [JUDUL] : No Title
   [ISI]   : Pedoman akademik > I. Nama Pimpinan : Rektor: Dr. I Nyoman Jampel, M.Pd.  
Wakil Rektor I: Prof. Dr. Ida Bagus Putu Arnyana, M.Si.  
Wakil Rektor II: Prof. Dr. I Wayan Lasmawan, M.Pd.  
Wakil Rektor III: Dr. I Gusti Ngurah Pujawan, M.Kes.  
Wakil Rektor IV: Drs. I Wayan Suarnajaya, M.A., Ph.D....

..............................

DOKUMEN KE-2
   [SKOR]  : 0.9949294328689575
   [JUDUL] : No Title
   [ISI]   : Pedoman akademik > 3.3.2. Teknologi Informasi dan Komunikasi : 

In [ ]:
#pertanyaan multihop ini gabisa pake rerangker bge large dan v2m3, harus 1 kalimat 1 kalimat, tapi pake embeding bisa meski skor nya kcil
q = "Jika seorang mahasiswa dari perguruan tinggi negeri lain ingin pindah ke Undiksha, dokumen apa saja yang harus dilampirkan dalam surat permohonan kepada Rektor, dan berapa jumlah SKS minimal yang harus sudah dia tempuh di perguruan tinggi asalnya?"
hasil = await query_rrf(q, rrf_retriever_chain)


🚀 PERTANYAAN: Jika seorang mahasiswa dari perguruan tinggi negeri lain ingin pindah ke Undiksha, dokumen apa saja yang harus dilampirkan dalam surat permohonan kepada Rektor, dan berapa jumlah SKS minimal yang harus sudah dia tempuh di perguruan tinggi asalnya?

[THINKING] Memulai proses retrieval untuk: 'Jika seorang mahasiswa dari perguruan tinggi negeri lain ingin pindah ke Undiksha, dokumen apa saja yang harus dilampirkan dalam surat permohonan kepada Rektor, dan berapa jumlah SKS minimal yang harus sudah dia tempuh di perguruan tinggi asalnya?'
[THINKING] Menghasilkan dense & sparse embeddings...
[THINKING] Melakukan Hybrid Search (RRF) di Qdrant...
[THINKING] Berhasil mengambil 20 dokumen kandidat dari RRF.
[THINKING] Memulai reranking dengan BAAI/bge-reranker-large...
Rank 1 | Score=0.9880 | Title=None
Rank 2 | Score=0.9161 | Title=None
Rank 3 | Score=0.6230 | Title=None
Reranking selesai. Mengembalikan Top 3 dokumen.

💡 JAWABAN:
Mahasiswa tersebut harus melampirkan Kutipan Daf

Membuat index di qdrant

In [1]:
from qdrant_client import QdrantClient, models
QDRANT_PATH = "./qdrant_custom_db_qwen"
COLLECTION_NAME = "hybrid_qwen_splade_optimized"
client = QdrantClient(path=QDRANT_PATH)

# Ambil 1 sample data dari koleksi
sample_data = client.scroll(
    collection_name=COLLECTION_NAME,
    limit=1
)

if sample_data[0]:
    # Lihat semua kunci (keys) yang ada di payload
    payload_keys = sample_data[0][0].payload.keys()
    print(f"Kunci payload yang ditemukan: {list(payload_keys)}")
    
    # Cetak isi payload untuk memastikan mana yang berisi teks utama
    print("\nContoh isi payload:")
    print(sample_data[0][0].payload)
else:
    print("Koleksi kosong atau tidak ditemukan.")

Kunci payload yang ditemukan: ['text', 'original_content', 'metadata']

Contoh isi payload:
{'text': 'Efisiensi KL TA 2025 > REPUBLIK INDONESIA : **Nomor** : S-37/MK.02/2025\n**Sifat** : Segera\n**Lampiran** : 1 (satu) berkas\n**Hal** : Efisiensi Belanja Kementerian/Lembaga dalam Pelaksanaan Anggaran Pendapatan Dan Belanja Negara Tahun Anggaran 2025  \n**24 Januari 2025**  \n**Yth.**\n1. Para Menteri Kabinet Merah Putih;\n2. Kepala Kepolisian Negara Republik Indonesia;\n3. Jaksa Agung Republik Indonesia;\n4. Para Kepala Lembaga Pemerintah Non Kementerian;\n5. Para Pimpinan Kesekretariatan Lembaga Negara.  \nDalam rangka pelaksanaan Undang-Undang Nomor 62 Tahun 2024 tentang Anggaran Pendapatan dan Belanja Negara TA 2025, disampaikan hal-hal sebagai berikut:  \n1. Sesuai Instruksi Presiden Republik Indonesia Nomor 1 Tahun 2025 tentang Efisiensi Belanja Dalam Pelaksanaan Anggaran Pendapatan Dan Belanja Negara Dan Anggaran Pendapatan Dan Belanja Daerah Tahun Anggaran 2025, menginstruksikan

In [2]:
from qdrant_client import models
from qdrant_client import QdrantClient, models
QDRANT_PATH = "./qdrant_custom_db_qwen"
COLLECTION_NAME = "hybrid_qwen_splade_optimized"
client = QdrantClient(path=QDRANT_PATH)

client.create_payload_index(
    collection_name=COLLECTION_NAME,
    field_name="text", # Sesuai hasil print kamu
    field_schema=models.TextIndexParams(
        type="text",
        tokenizer=models.TokenizerType.WORD,
        lowercase=True,
    ),
)
print("✅ Payload index untuk field 'text' berhasil dibuat!")

✅ Payload index untuk field 'text' berhasil dibuat!


C:\Users\Ilmu Komputer\AppData\Local\Temp\ipykernel_1944\1984171025.py:7: UserWarning: Payload indexes have no effect in the local Qdrant. Please use server Qdrant if you need payload indexes.
  client.create_payload_index(


Mendownload gemma 7b it

In [1]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-7b-it",
    device_map="auto",
    torch_dtype=torch.float16,
    revision="float16",
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))


`torch_dtype` is deprecated! Use `dtype` instead!


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

RuntimeError: Data processing error: CAS service error : ReqwestMiddleware Error: couldn't get token: TokenRefreshFailure("Error refreshing token: PyErr { type: <class 'requests.exceptions.ReadTimeout'>, value: ReadTimeout(ReadTimeoutError(\"HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=None)\"), '(Request ID: 6fa4ba6d-cfd0-4bf9-ba7a-d3159f12768d)'), traceback: Some(\"Traceback (most recent call last):\\n  File \\\"c:\\\\conda\\\\miniconda3\\\\envs\\\\defi\\\\Lib\\\\site-packages\\\\huggingface_hub\\\\file_download.py\\\", line 594, in token_refresher\\n    connection_info = refresh_xet_connection_info(file_data=xet_file_data, headers=headers)\\n                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\\n  File \\\"c:\\\\conda\\\\miniconda3\\\\envs\\\\defi\\\\Lib\\\\site-packages\\\\huggingface_hub\\\\utils\\\\_validators.py\\\", line 114, in _inner_fn\\n    return fn(*args, **kwargs)\\n           ^^^^^^^^^^^^^^^^^